# Vector Search + EMI Calculator: Complete Execution Patterns

Combining RAG-based vehicle search with financial calculations to demonstrate parallel, sequential, and conversational execution patterns.

## Learning Objectives

By the end of this notebook, you will:

1. **Integrate RAG with computational tools** - Combine vector similarity search with financial calculations
2. **Execute parallel workflows** - Handle independent tasks (search + calculation) simultaneously
3. **Execute sequential workflows** - Chain dependent tasks where search results inform calculations
4. **Manage conversational context** - Build multi-turn conversations for complex car buying scenarios

## 1. Environment Setup

In [1]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

from dotenv import load_dotenv
from typing import Literal, List

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [2]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized")

✅ LLM initialized


## 2. Setup Vector Store

In [3]:
# ChromaDB Configuration
PERSIST_DIR = "../../chroma_db"
COLLECTION_NAME = "toyota_specs"
EMBED_MODEL_ID = "gemini-embedding-001"

# Initialize embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(
    model=EMBED_MODEL_ID,
    output_dimensionality=768
)

# Connect to vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings_model,
    persist_directory=PERSIST_DIR
)

print(f"✅ Connected to vectorstore: {COLLECTION_NAME}")
print(f"   Total documents: {vectorstore._collection.count()}")

/var/folders/n4/66bq258d6xq6lscwjlgs6q500000gn/T/ipykernel_10483/4137707577.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Connected to vectorstore: toyota_specs
   Total documents: 32


## 3. Define Tools

In [4]:
# Vector similarity search helper
def vector_similarity_search(
    query: str, 
    vectorstore, 
    k: int = 5
) -> List[str]:
    """Perform vector similarity search."""
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

In [5]:
# Tool 1: Vehicle Search
@tool
def search_vehicles(query: str, max_results: int = 5) -> str:
    """
    Search Toyota vehicle database using semantic similarity.
    
    Use this tool when users need information about Toyota vehicles,
    including specifications, pricing, fuel efficiency, or comparisons.
    
    Args:
        query: Natural language search query about Toyota vehicles
        max_results: Maximum number of results to return (default: 5)
    
    Returns:
        Formatted string with vehicle information
    """
    docs = vector_similarity_search(query, vectorstore, k=max_results)
    
    result = "Vehicle Search Results:\n"
    result += "=" * 60 + "\n"
    for i, doc in enumerate(docs, 1):
        result += f"\nResult {i}:\n{doc}\n"
    result += "=" * 60
    
    return result

print("✅ search_vehicles tool defined")

✅ search_vehicles tool defined


In [6]:
# Tool 2: EMI Calculator
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5)
        tenure_months: Loan tenure in months
        currency: Currency code (USD, EUR, GBP, INR, JPY)
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ emi_calculator tool defined")

✅ emi_calculator tool defined


## 4. Build Graph

In [7]:
# Initialize LLM with tools
tools = [search_vehicles, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


## 5. Test Case 1: Parallel Execution

**Independent Tasks**: Search for fuel-efficient sedans AND calculate EMI for a $25,000 loan.

These tasks have no dependencies - they can run simultaneously.

In [8]:
# Parallel test: Independent tasks
parallel_state = {
    "messages": [
        HumanMessage(content="Find me fuel-efficient Toyota sedans AND ALSO calculate the EMI for a 25000 USD loan at 6.5% for 60 months")
    ]
}

print("TEST CASE 1: PARALLEL EXECUTION")
print("=" * 80)
print("Query: Find fuel-efficient sedans AND calculate EMI for $25,000")
print("\nExpected: SINGLE AIMessage with MULTIPLE tool_calls")
print("=" * 80)

TEST CASE 1: PARALLEL EXECUTION
Query: Find fuel-efficient sedans AND calculate EMI for $25,000

Expected: SINGLE AIMessage with MULTIPLE tool_calls


In [9]:
# Execute parallel test
parallel_result = app.invoke(parallel_state)
print(f"\nTotal messages: {len(parallel_result['messages'])}")


Total messages: 5


In [10]:
# Verify parallel execution
tool_call_message = parallel_result['messages'][1]

print("\nPARALLEL EXECUTION VERIFICATION")
print("=" * 80)
print(f"Number of tool_calls in AIMessage: {len(tool_call_message.tool_calls)}")

if len(tool_call_message.tool_calls) > 1:
    print(f"\n🚀 CONFIRMED: Parallel execution!")
    print(f"   {len(tool_call_message.tool_calls)} tools called in a SINGLE AIMessage\n")
    
    for i, tc in enumerate(tool_call_message.tool_calls, 1):
        print(f"  Tool {i}: {tc['name']}")
        print(f"    Args: {tc['args']}\n")
else:
    print("\n⚠️ Sequential execution detected (not parallel)")


PARALLEL EXECUTION VERIFICATION
Number of tool_calls in AIMessage: 2

🚀 CONFIRMED: Parallel execution!
   2 tools called in a SINGLE AIMessage

  Tool 1: search_vehicles
    Args: {'query': 'fuel-efficient Toyota sedans'}

  Tool 2: emi_calculator
    Args: {'annual_interest_rate': 6.5, 'tenure_months': 60, 'principal': 25000, 'currency': 'USD'}



In [11]:
# Show final response
print("FINAL RESPONSE:")
print("=" * 80)
print(parallel_result['messages'][-1].content)
print("=" * 80)

FINAL RESPONSE:
Here are some fuel-efficient Toyota sedans based on your search:

*   **Toyota Camry:** Gasoline Engine: \~28 MPG (city) / \~39 MPG (highway), Hybrid Powertrain: \~51 MPG (city) / \~53 MPG (highway)
*   **Toyota Prius (Hybrid):** \~58 MPG (city) / \~53 MPG (highway)
*   **Toyota Prius Prime (Plug-in Hybrid):** \~133 MPGe (electric + hybrid) / \~42 miles all-electric range
*   **Toyota Corolla:** Gasoline Engine: 30 MPG (city) / 38 MPG (highway), Hybrid Engine: 53 MPG (city) / 52 MPG (highway)

The EMI for a 25000 USD loan at 6.5% for 60 months is:

*   **Monthly EMI:** 489.15 USD
*   **Total Payment:** 29,349.22 USD
*   **Total Interest:** 4,349.22 USD


## 6. Test Case 2: Sequential Execution

**Dependent Tasks**: Find the cheapest Camry trim, THEN calculate EMI for that price.

The EMI calculation depends on the search result.

In [12]:
# Sequential test: Dependent tasks
sequential_state = {
    "messages": [
        HumanMessage(content="Find the base price of the Toyota Camry, then calculate the EMI for that amount in USD at 7% interest for 60 months")
    ]
}

print("TEST CASE 2: SEQUENTIAL EXECUTION")
print("=" * 80)
print("Query: Find Camry base price, THEN calculate its EMI")
print("\nExpected: MULTIPLE SEPARATE AIMessages with tool_calls")
print("=" * 80)

TEST CASE 2: SEQUENTIAL EXECUTION
Query: Find Camry base price, THEN calculate its EMI

Expected: MULTIPLE SEPARATE AIMessages with tool_calls


In [13]:
# Execute sequential test
sequential_result = app.invoke(sequential_state)
print(f"\nTotal messages: {len(sequential_result['messages'])}")


Total messages: 6


In [14]:
# Verify sequential execution
tool_call_messages = [
    msg for msg in sequential_result['messages']
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls
]

print("SEQUENTIAL EXECUTION VERIFICATION")
print("=" * 80)
print(f"AIMessages with tool_calls: {len(tool_call_messages)}")

if len(tool_call_messages) > 1:
    print("\n🔄 CONFIRMED: Sequential execution detected!")
    print(f"   {len(tool_call_messages)} SEPARATE tool call requests\n")
    
    for i, msg in enumerate(tool_call_messages, 1):
        print(f"  Loop {i}: {msg.tool_calls[0]['name']}")
        print(f"    Args: {msg.tool_calls[0]['args']}\n")
else:
    print("\n⚠️ Single tool call message (might be parallel)")

SEQUENTIAL EXECUTION VERIFICATION
AIMessages with tool_calls: 2

🔄 CONFIRMED: Sequential execution detected!
   2 SEPARATE tool call requests

  Loop 1: search_vehicles
    Args: {'query': 'base price of Toyota Camry'}

  Loop 2: emi_calculator
    Args: {'principal': 26000, 'tenure_months': 60, 'currency': 'USD', 'annual_interest_rate': 7}



In [15]:
# Verify data flow
print("DATA FLOW VERIFICATION")
print("=" * 80)

if len(tool_call_messages) >= 2:
    # Get the vehicle search result
    search_result = sequential_result['messages'][2]  # First ToolMessage
    print("STEP 1 - Vehicle Search Result:")
    print(f"  {search_result.content[:200]}...\n")
    
    # Get the EMI tool call
    emi_call = sequential_result['messages'][3]  # Second AIMessage
    print("STEP 2 - EMI Calculator Called With:")
    for key, value in emi_call.tool_calls[0]['args'].items():
        print(f"  {key}: {value}")
    
    print("\n" + "-" * 80)
    print("✅ LLM extracted price from search and used it for EMI calculation!")

DATA FLOW VERIFICATION
STEP 1 - Vehicle Search Result:
  Vehicle Search Results:

Result 1:
Pricing
Base Model (LE): Starting at $26,000
Mid-Level Trim (SE): Starting at $28,000
Premium Trim (XSE ...

STEP 2 - EMI Calculator Called With:
  principal: 26000
  tenure_months: 60
  currency: USD
  annual_interest_rate: 7

--------------------------------------------------------------------------------
✅ LLM extracted price from search and used it for EMI calculation!


In [16]:
# Show final response
print("\nFINAL RESPONSE:")
print("=" * 80)
print(sequential_result['messages'][-1].content)
print("=" * 80)


FINAL RESPONSE:
The base price of the Toyota Camry (LE) is $26,000. The EMI for that amount at 7% interest for 60 months would be $514.83. The total payment would be $30,889.87, with a total interest of $4,889.87.


## 7. Test Case 3: Conversational Context

**Multi-Turn Conversation**: Car buying journey with context maintained across turns.

In [17]:
# Initialize conversation
conversation = {"messages": []}

print("TEST CASE 3: CONVERSATIONAL CONTEXT")
print("=" * 80)
print("Scenario: Multi-turn car buying conversation\n")

TEST CASE 3: CONVERSATIONAL CONTEXT
Scenario: Multi-turn car buying conversation



### Turn 1: Initial Vehicle Search

In [18]:
# Turn 1: Find vehicles
print("TURN 1")
print("─" * 80)

user_query_1 = "I'm looking for an affordable Toyota SUV with good fuel economy. What do you recommend?"
print(f"👤 User: {user_query_1}")

conversation["messages"].append(HumanMessage(content=user_query_1))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

TURN 1
────────────────────────────────────────────────────────────────────────────────
👤 User: I'm looking for an affordable Toyota SUV with good fuel economy. What do you recommend?

🤖 Assistant: Based on the search results, the Toyota RAV4 and Toyota Prius are both affordable SUVs with good fuel economy. The RAV4 has hybrid and plug-in hybrid options, while the Prius is known for its excellent fuel efficiency and is available as a hybrid or plug-in hybrid.

📊 State size: 4 messages


### Turn 2: EMI Calculation for Mentioned Vehicle

In [19]:
# Turn 2: Calculate EMI for recommended vehicle
print("\n" + "─" * 80)
print("TURN 2")
print("─" * 80)

user_query_2 = "What would the monthly payment be for that RAV4 at 6.5% interest over 5 years?"
print(f"👤 User: {user_query_2}")
print("\n   Note: 'that RAV4' references Turn 1's recommendation!")

conversation["messages"].append(HumanMessage(content=user_query_2))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")


────────────────────────────────────────────────────────────────────────────────
TURN 2
────────────────────────────────────────────────────────────────────────────────
👤 User: What would the monthly payment be for that RAV4 at 6.5% interest over 5 years?

   Note: 'that RAV4' references Turn 1's recommendation!

🤖 Assistant: Could you please specify the price of the RAV4 you are interested in? There are different trim levels and options, which affect the price. Once I have the price, I can calculate the monthly payment for you.

📊 State size: 6 messages


### Turn 3: Alternative Comparison

In [20]:
# Turn 3: Compare with alternative
print("\n" + "─" * 80)
print("TURN 3")
print("─" * 80)

user_query_3 = "What about the Highlander? How does its monthly payment compare?"
print(f"👤 User: {user_query_3}")
print("\n   Note: Comparing with previous EMI calculation")

conversation["messages"].append(HumanMessage(content=user_query_3))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")


────────────────────────────────────────────────────────────────────────────────
TURN 3
────────────────────────────────────────────────────────────────────────────────
👤 User: What about the Highlander? How does its monthly payment compare?

   Note: Comparing with previous EMI calculation

🤖 Assistant: Okay, let's compare the monthly payments. I'll use the base prices for both vehicles and a 6.5% interest rate over 5 years (60 months).

*   **Toyota RAV4 (Base Model - $28,000):** The estimated monthly payment would be $547.85.
*   **Toyota Highlander (Base Model - $37,000):** The estimated monthly payment would be $723.95.

As you can see, the Highlander's monthly payment is significantly higher than the RAV4's due to its higher base price.

📊 State size: 13 messages


### Turn 4: Memory-Based Question

In [21]:
# Turn 4: Recall previous calculation
print("\n" + "─" * 80)
print("TURN 4")
print("─" * 80)

user_query_4 = "Remind me, what was the monthly payment difference between those two?"
print(f"👤 User: {user_query_4}")
print("\n   Note: Agent should answer from memory, no tool needed!")

conversation["messages"].append(HumanMessage(content=user_query_4))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")


────────────────────────────────────────────────────────────────────────────────
TURN 4
────────────────────────────────────────────────────────────────────────────────
👤 User: Remind me, what was the monthly payment difference between those two?

   Note: Agent should answer from memory, no tool needed!

🤖 Assistant: The monthly payment for the Toyota RAV4 (Base Model - $28,000) is $547.85, and the monthly payment for the Toyota Highlander (Base Model - $37,000) is $723.95. The difference in monthly payments is $176.10.

📊 State size: 15 messages


### Conversation State Analysis

In [22]:
# Examine conversation state
print("\nCOMPLETE CONVERSATION STATE")
print("=" * 80)

turn_num = 0
for i, msg in enumerate(conversation["messages"], 1):
    if isinstance(msg, HumanMessage):
        turn_num += 1
        print(f"\n{'─' * 80}")
        print(f"TURN {turn_num}")
        print(f"{'─' * 80}")
        preview = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
        print(f"[{i}] 👤 USER: {preview}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            tools_called = ", ".join([tc['name'] for tc in msg.tool_calls])
            print(f"[{i}] 🤖 AGENT: Calling {tools_called}")
        else:
            preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{i}] 🤖 AGENT: {preview}")
            
    elif isinstance(msg, ToolMessage):
        first_line = msg.content.split('\n')[0]
        print(f"[{i}] 🔧 TOOL: {first_line}")

print(f"\n{'=' * 80}")
print(f"Total messages in state: {len(conversation['messages'])}")

# Count message types
human_count = sum(1 for m in conversation["messages"] if isinstance(m, HumanMessage))
ai_count = sum(1 for m in conversation["messages"] if isinstance(m, AIMessage))
tool_count = sum(1 for m in conversation["messages"] if isinstance(m, ToolMessage))

print(f"\nMessage breakdown:")
print(f"  👤 HumanMessages:  {human_count}")
print(f"  🤖 AIMessages:     {ai_count}")
print(f"  🔧 ToolMessages:   {tool_count}")


COMPLETE CONVERSATION STATE

────────────────────────────────────────────────────────────────────────────────
TURN 1
────────────────────────────────────────────────────────────────────────────────
[1] 👤 USER: I'm looking for an affordable Toyota SUV with good fuel econ...
[2] 🤖 AGENT: Calling search_vehicles
[3] 🔧 TOOL: Vehicle Search Results:
[4] 🤖 AGENT: Based on the search results, the Toyota RAV4 and T...

────────────────────────────────────────────────────────────────────────────────
TURN 2
────────────────────────────────────────────────────────────────────────────────
[5] 👤 USER: What would the monthly payment be for that RAV4 at 6.5% inte...
[6] 🤖 AGENT: Could you please specify the price of the RAV4 you...

────────────────────────────────────────────────────────────────────────────────
TURN 3
────────────────────────────────────────────────────────────────────────────────
[7] 👤 USER: What about the Highlander? How does its monthly payment comp...
[8] 🤖 AGENT: Calling searc

## 8. Execution Pattern Comparison

In [23]:
# Compare execution patterns
print("EXECUTION PATTERN COMPARISON")
print("=" * 80)

print("\n1. PARALLEL EXECUTION:")
print("   Pattern: Single AIMessage with multiple tool_calls")
print(f"   Message count: {len(parallel_result['messages'])}")
print(f"   Tool calls in AIMessage: {len(parallel_result['messages'][1].tool_calls)}")
print("   Use case: Independent tasks (search + calculate)")

print("\n2. SEQUENTIAL EXECUTION:")
print("   Pattern: Multiple separate AIMessages with tool_calls")
print(f"   Message count: {len(sequential_result['messages'])}")
tool_call_msgs = [m for m in sequential_result['messages'] 
                  if isinstance(m, AIMessage) and hasattr(m, 'tool_calls') and m.tool_calls]
print(f"   Number of tool call loops: {len(tool_call_msgs)}")
print("   Use case: Dependent tasks (search result → EMI input)")

print("\n3. CONVERSATIONAL CONTEXT:")
print("   Pattern: State accumulates across multiple user turns")
print(f"   Message count: {len(conversation['messages'])}")
print(f"   Number of turns: {human_count}")
print("   Use case: Multi-turn car buying journey")

print("\n" + "=" * 80)

EXECUTION PATTERN COMPARISON

1. PARALLEL EXECUTION:
   Pattern: Single AIMessage with multiple tool_calls
   Message count: 5
   Tool calls in AIMessage: 2
   Use case: Independent tasks (search + calculate)

2. SEQUENTIAL EXECUTION:
   Pattern: Multiple separate AIMessages with tool_calls
   Message count: 6
   Number of tool call loops: 2
   Use case: Dependent tasks (search result → EMI input)

3. CONVERSATIONAL CONTEXT:
   Pattern: State accumulates across multiple user turns
   Message count: 15
   Number of turns: 4
   Use case: Multi-turn car buying journey



## Conclusion

In this notebook, you learned:

✅ **RAG + Computational Tools** - Combined vector similarity search (RAG) with financial calculations (EMI) to create a practical car-buying assistant that answers both informational and computational queries

✅ **Parallel Execution** - When tasks are independent (e.g., "find sedans AND calculate EMI"), the LLM calls both tools simultaneously in a single AIMessage, resulting in faster execution (5 messages vs 6+)

✅ **Sequential Execution** - When tasks have dependencies (e.g., "find price, THEN calculate EMI"), the LLM executes tools in separate loops, using results from search_vehicles to populate emi_calculator parameters

✅ **Conversational Context** - State accumulates across turns (4 user queries → 14+ messages), enabling the LLM to reference previous searches ("that RAV4") and calculations ("those two") for natural multi-turn conversations

### Key Insights

**Pattern Recognition**: The LLM automatically determines execution strategy based on query structure - "AND ALSO" triggers parallel, "then" triggers sequential, and multi-turn naturally maintains context

**Real-World Application**: This pattern (search database + calculate finances) applies to many domains - e-commerce (product search + price calculator), real estate (property search + mortgage calculator), or travel (flight search + cost estimator)

**Message Efficiency**: Parallel execution (5 messages) is more efficient than sequential (6+ messages) when tasks are independent, but sequential is necessary when one task depends on another's result

### Practical Applications

This notebook demonstrates a complete RAG-powered assistant for:
- Car dealership chatbots
- E-commerce platforms with financing options
- Real estate websites with mortgage calculators
- Any domain combining information retrieval with financial calculations

### Series Complete!

Congratulations! You've completed the comprehensive LangGraph tutorial series covering:
- **Notebooks 01-02**: Setup and tool creation fundamentals
- **Notebooks 03-05**: Practical tool development (currency, EMI, graph construction)
- **Notebooks 06-09**: Execution patterns (single, parallel, sequential, conversational)
- **Notebooks 10-12**: RAG integration with vector similarity search
- **Notebook 13**: Complete real-world application combining all concepts

You're now equipped to build production-ready agentic workflows with LangGraph!